In [1]:
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.01
set_session(tf.Session(config=config))

from keras.models import load_model
PATH_MODEL = "../Models/LUNA_model_v2_2.h5"
model = load_model(PATH_MODEL)

Using TensorFlow backend.
/opt/anaconda/lib/python3.5/site-packages/keras/engine/topology.py:1206: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  return cls(**config)


In [2]:
import numpy as np
import pandas as pd
import os
import time

PATH_VOXELS = '../../data/stage1_voxels_mask/'
VOXEL_SIZE = 64

In [21]:
def feature_vect(patient):     
    import time
    patient_array = np.load(PATH_VOXELS + patient)
    voxels = patient_array['vox']  
    %time np.array(model_v24.predict(x= voxels))
    preds = np.array(model_v24.predict(x= voxels))
    ixs = np.argmax(preds[0])
    
    xmax_malig = np.max(preds[0], axis=0)
    xmax_spiculation = np.max(preds[1], axis=0)
    xmax_lobulation = np.max(preds[2], axis=0)
    xmax_diameter = np.max(preds[3], axis=0)
    
    xsd_malig = np.std(preds[0], axis=0)
    xsd_spiculation = np.std(preds[1], axis=0)
    xsd_lobulation = np.std(preds[2], axis=0)
    xsd_diameter = np.std(preds[3], axis=0)
    
#     locs = patient_array['locs']
    centroids = patient_array['cents']
    shape = patient_array['shape']
    normalized_locs = centroids.astype('float32') / shape.astype('float32')
    
    feats = (np.concatenate([xmax_malig,xmax_spiculation,xmax_lobulation,xmax_diameter,\
               xsd_malig,xsd_spiculation,xmax_lobulation,xsd_diameter,\
               normalized_locs[ixs],normalized_locs.std(axis=0)]))        
    return feats

In [22]:
# unit test
start = time.time()
feats = feature_vect(patient = '008464bb8521d09a42985dd8add3d0d2.npz')
print ("It took %.2d s" %(time.time()-start))
feats

CPU times: user 8.2 s, sys: 1.04 s, total: 9.24 s
Wall time: 46.5 s
It took 94 s


array([ 0.63497519,  0.34936088,  0.39840066,  0.3574523 ,  0.15504055,
        0.08605267,  0.39840066,  0.08193393,  0.35945946,  0.39729729,
        0.53082192,  0.20804369,  0.15417519,  0.23205459], dtype=float32)

In [5]:
%time 2+2

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 18.4 µs


4

In [3]:
start = time.time()
patients = [f for f in os.listdir(PATH_VOXELS)]
print ("patient numbers: ", len(patients))

all_features = []
for num, patient in enumerate(patients):
    
    patient_array = np.load(PATH_VOXELS + patient)
    voxels = patient_array['vox']  
    preds = np.array(model.predict(x= voxels))
    
df = pd.DataFrame(data=preds,index=patients)
df.to_csv('./model2_predict.csv')

patient numbers:  1434


ResourceExhaustedError: OOM when allocating tensor with shape[32,64,64,64,8]
	 [[Node: conv3d_1/convolution = Conv3D[T=DT_FLOAT, padding="SAME", strides=[1, 1, 1, 1, 1], _device="/job:localhost/replica:0/task:0/gpu:0"](conv3d_1/transpose, conv3d_1/kernel/read)]]
	 [[Node: batch_normalization_2/moments/sufficient_statistics/Gather/_657 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/cpu:0", send_device="/job:localhost/replica:0/task:0/gpu:0", send_device_incarnation=1, tensor_name="edge_260_batch_normalization_2/moments/sufficient_statistics/Gather", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/cpu:0"]()]]

Caused by op 'conv3d_1/convolution', defined at:
  File "/opt/anaconda/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/opt/anaconda/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/opt/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py", line 3, in <